# Preamble

In [1]:
import xarray as xr
import numpy as np
import holoviews as hv
from pathlib import Path
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np
from datetime import date
from matplotlib import pyplot as plt
# %matplotlib widget
from holoviews import opts
import dask.array as da
from dask.distributed import Client, LocalCluster
import param
from holoviews import streams
# Get working directory
import sys, os
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import marineHeatWaves as mhw

# Import data

In [2]:
ds_lswt = xr.open_dataset('../../output/DINEOF_lake_dataset_with_clim.nc')
ds_lswt

<xarray.Dataset>
Dimensions:      (time: 5844, lat: 52, lon: 43)
Coordinates:
  * time         (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2022-12-31
  * lat          (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.86 45.87
  * lon          (lon) float64 10.51 10.52 10.53 10.54 ... 10.85 10.85 10.86
Data variables:
    lswt         (time, lat, lon) float32 ...
    clim         (time, lat, lon) float64 ...
    thresh       (time, lat, lon) float64 ...
    intensity    (time, lat, lon) float64 ...
    category     (time, lat, lon) float64 ...
    extrathresh  (time, lat, lon) float64 ...

# Methodology

In [3]:
da_intensity=ds_lswt.intensity
da_category=ds_lswt.category

## Caculate HWs per each point and create a mask

In [4]:
da_category.shape

(5844, 52, 43)

In [5]:
# Initialize the mask with False values
mask = np.zeros(da_category.shape[:], dtype=bool)

# Iterate over each point
for lat in range(da_category.shape[1]):
    for lon in range(da_category.shape[2]):
        # Initialize counters
        consecutive_days = 0
        start_index = None

        # Iterate over each time step within the specified range
        for t in range(da_category.shape[0]):
            # Check if intensity is greater than 0
            if da_category[t, lat, lon] > 0.5:
                if consecutive_days == 0:
                    start_index = t
                consecutive_days += 1
                # Check if intensity is greater than 0 for 5 continuous days
                if consecutive_days >= 5:
                    # Mark the series of at least 5 consecutive days as True
                    end_index = start_index + consecutive_days - 1
                    mask[start_index:end_index, lat, lon] = True
            else:
                # Reset the consecutive days counter if intensity is not greater than 0
                consecutive_days = 0
                start_index = None

# Iterate again to check for gaps between the series
for lat in range(da_category.shape[1]):
    for lon in range(da_category.shape[2]):
        # Initialize variables
        gap_count = 0

        for t in range(da_category.shape[0]):
            # Check if the current day is part of a series
            if mask[t, lat, lon]:
                # If there was a gap and it's less than 2 days, mark the gap days as True
                if gap_count > 0 and gap_count <= 2:
                    mask[t - gap_count:t, lat, lon] = True
                gap_count = 0
            else:
                # Increment the gap count
                gap_count += 1

# Now, 'mask' contains True where the condition is satisfied within the specified time interval


In [6]:
# Create a new xarray dataset with the mask array
ds_mask = xr.DataArray(mask, dims=da_category.dims, coords=da_category.coords)

# Save the dataset to a NetCDF file
ds_mask.to_netcdf('..\..\output\DINEOF_HWs_mask_per_cell.nc')